## Day 46 Lecture 1 Assignment

In this assignment, we will calculate a distance matrix for geographical Starbucks data and use it to identify locations that are close together and far apart. We will perform clustering on this dataset later on.

We will be using the "haversine" package to compute geographical distance. It can be pip installed.

In [1]:
# !pip install haversine

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from haversine import haversine

Below is a convenience function for calculating geographical distance matrices using lat-long data.

In [3]:
def geo_sim_matrix(df, col_name = 'Coordinates'):
    """
    A function that computes a geographical distance matrix (in miles).
    Each row in the dataframe should correspond to one location.
    In addition, the dataframe must have a column containing the lat-long of each location as a tuple (i.e. (lat, long)).
    
    Parameters:
        df (pandas dataframe): an nxm dataframe containing the locations to compute similarities between.
        col (string): the name of the column containing the lat-long tuples.
        
    Returns:
        distance (pandas dataframe): an nxn distance matrix between the geographical coordinates of each location.
    """
    
    df = df.copy()
    df.reset_index(inplace=True)
    haver_vec = np.vectorize(haversine, otypes=[np.float32])
    distance = df.groupby('index').apply(lambda x: pd.Series(haver_vec(df[col_name], x[col_name])))
    distance = distance / 1.609344  # converts to miles
    distance.columns = distance.index
    
    return distance


This dataset contains the latitude and longitude (as well as several other details we will not be using) of every Starbucks in the world as of February 2017. Each row consists of the following features, which are generally self-explanatory:

- Brand
- Store Number
- Store Name
- Ownership Type
- Street Address
- City
- State/Province
- Country
- Postcode
- Phone Number
- Timezone
- Longitude
- Latitude

Load in the dataset.

In [4]:
# answer goes here
df = pd.read_csv('data/starbucks_locations.csv')

In [8]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25600 entries, 0 to 25599
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Brand           25600 non-null  object 
 1   Store Number    25600 non-null  object 
 2   Store Name      25600 non-null  object 
 3   Ownership Type  25600 non-null  object 
 4   Street Address  25598 non-null  object 
 5   City            25585 non-null  object 
 6   State/Province  25600 non-null  object 
 7   Country         25600 non-null  object 
 8   Postcode        24078 non-null  object 
 9   Phone Number    18739 non-null  object 
 10  Timezone        25600 non-null  object 
 11  Longitude       25599 non-null  float64
 12  Latitude        25599 non-null  float64
dtypes: float64(2), object(11)
memory usage: 2.5+ MB


,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51


In [12]:
df['Country'].unique()

array(['AD', 'AE', 'AR', 'AT', 'AU', 'AW', 'AZ', 'BE', 'BG', 'BH', 'BN',
       'BO', 'BR', 'BS', 'CA', 'CH', 'CL', 'CN', 'CO', 'CR', 'CW', 'CY',
       'CZ', 'DE', 'DK', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HU',
       'ID', 'IE', 'IN', 'JO', 'JP', 'KH', 'KR', 'KW', 'KZ', 'LB', 'LU',
       'MA', 'MC', 'MX', 'MY', 'NL', 'NO', 'NZ', 'OM', 'PA', 'PE', 'PH',
       'PL', 'PR', 'PT', 'QA', 'RO', 'RU', 'SA', 'SE', 'SG', 'SK', 'SV',
       'TH', 'TR', 'TT', 'TW', 'US', 'VN', 'ZA'], dtype=object)

Begin by narrowing down the dataset to a specific geographic area of interest. Since we will need to manually compute a distance matrix, which will be on the order of $n^{2}$ in terms of size, we would recommend choosing an area with 3000 or less locations. In this example, we will use Hawaii, which has about 100 locations; for reference, California has about 2800 locations. Feel free to choose a different region that is of more interest to you, if desired.

Subset the dataframe to only include records for Starbucks locations in Hawaii.

In [14]:
# answer goes here
hi_df = df[df['State/Province'] == 'HI']
#hi_df = hi_df[ha_df['Country'] == 'US']
hi_df




,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
17202,Starbucks,21034-73360,Aiea Shopping Center,Company Owned,99-115 Aiea Heights Drive #125,Aiea,HI,US,967013913,808-484-1488,GMT-10:00 Pacific/Honolulu,-157.93,21.38
17203,Starbucks,21053-99755,Stadium Marketplace,Company Owned,4561 Salt Lake Boulevard,Aiea,HI,US,968183167,808-488-9313,GMT-10:00 Pacific/Honolulu,-157.93,21.37
17204,Starbucks,21006-10033,Kaonohi St & Kam Hwy - Pearlridge,Company Owned,98-125 Kaonohi Street,Aiea,HI,US,967012318,808-484-9548,GMT-10:00 Pacific/Honolulu,-157.94,21.38
17205,Starbucks,21005-10034,Pearlridge Mall Uptown,Company Owned,98-1005 Moanalua Road,Aiea,HI,US,967014705,808-484-9355,GMT-10:00 Pacific/Honolulu,-157.94,21.39
17206,Starbucks,21063-101700,Waimalu Shopping Center,Company Owned,"98-1277 Kaahumanu Street, Building E, Unit 7, ...",Aiea,HI,US,967015314,808-484-5802,GMT-10:00 Pacific/Honolulu,-157.95,21.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17296,Starbucks,70063-139304,Wailea Beach Resort - Marriott Maui,Licensed,3700 Wailea Alanui Dr,Wailea,HI,US,967538347,808-874-7981,GMT-10:00 Pacific/Honolulu,-156.44,20.69
17297,Starbucks,19214-196545,Safeway - Wailuku 3092,Licensed,"58 Maui lani Pkwy, Waikele Center",Wailuku,HI,US,96793,808-243-3522,GMT-10:00 Pacific/Honolulu,-156.49,20.89
17298,Starbucks,21044-88761,Waikele Premium Outlets,Company Owned,"94-799 Lumiaina Street, Laniakea Plaza",Waipahu,HI,US,967975041,808-678-3418,GMT-10:00 Pacific/Honolulu,-158.01,21.40
17299,Starbucks,21061-99913,Laniakea Plaza at Ka Uka Blvd,Company Owned,"94-1221 Ka Uka Boulevard, Unit A-101",Waipahu,HI,US,967976202,808-680-9213,GMT-10:00 Pacific/Honolulu,-158.00,21.43


The haversine package takes tuples with 2 numeric elements and interprets them as lat-long to calculate distance, so add a new column called "Coordinates" that converts the lat and long in each row into a tuple. In other words, the last two columns of the dataframe should initially look like this:

**Latitude, Longitude**  
39.14, -121.64  
34.13, -116.40  
...

After adding the new column, the last three columns should look like this:

**Latitude, Longitude, Coordinates**  
39.14, -121.64,   (39.14, -121.64)  
34.13, -116.40,   (34.13, -116.40)  
...

In [17]:
# answer goes here
hi_df['Coordinates'] = list(zip(hi_df['Latitude'], hi_df["Longitude"]))
hi_df



C:\Users\dgump\miniconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,Coordinates
17202,Starbucks,21034-73360,Aiea Shopping Center,Company Owned,99-115 Aiea Heights Drive #125,Aiea,HI,US,967013913,808-484-1488,GMT-10:00 Pacific/Honolulu,-157.93,21.38,"(21.38, -157.93)"
17203,Starbucks,21053-99755,Stadium Marketplace,Company Owned,4561 Salt Lake Boulevard,Aiea,HI,US,968183167,808-488-9313,GMT-10:00 Pacific/Honolulu,-157.93,21.37,"(21.37, -157.93)"
17204,Starbucks,21006-10033,Kaonohi St & Kam Hwy - Pearlridge,Company Owned,98-125 Kaonohi Street,Aiea,HI,US,967012318,808-484-9548,GMT-10:00 Pacific/Honolulu,-157.94,21.38,"(21.38, -157.94)"
17205,Starbucks,21005-10034,Pearlridge Mall Uptown,Company Owned,98-1005 Moanalua Road,Aiea,HI,US,967014705,808-484-9355,GMT-10:00 Pacific/Honolulu,-157.94,21.39,"(21.39, -157.94)"
17206,Starbucks,21063-101700,Waimalu Shopping Center,Company Owned,"98-1277 Kaahumanu Street, Building E, Unit 7, ...",Aiea,HI,US,967015314,808-484-5802,GMT-10:00 Pacific/Honolulu,-157.95,21.39,"(21.39, -157.95)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17296,Starbucks,70063-139304,Wailea Beach Resort - Marriott Maui,Licensed,3700 Wailea Alanui Dr,Wailea,HI,US,967538347,808-874-7981,GMT-10:00 Pacific/Honolulu,-156.44,20.69,"(20.69, -156.44)"
17297,Starbucks,19214-196545,Safeway - Wailuku 3092,Licensed,"58 Maui lani Pkwy, Waikele Center",Wailuku,HI,US,96793,808-243-3522,GMT-10:00 Pacific/Honolulu,-156.49,20.89,"(20.89, -156.49)"
17298,Starbucks,21044-88761,Waikele Premium Outlets,Company Owned,"94-799 Lumiaina Street, Laniakea Plaza",Waipahu,HI,US,967975041,808-678-3418,GMT-10:00 Pacific/Honolulu,-158.01,21.40,"(21.4, -158.01)"
17299,Starbucks,21061-99913,Laniakea Plaza at Ka Uka Blvd,Company Owned,"94-1221 Ka Uka Boulevard, Unit A-101",Waipahu,HI,US,967976202,808-680-9213,GMT-10:00 Pacific/Honolulu,-158.00,21.43,"(21.43, -158.0)"


Calculate the distance matrix using the starter code/function geo_sim_matrix() provided earlier in the notebook. It assumes the column containing the coordinates for each location is called "Coordinates". Examine the docstring for more details.

Note: the latitude and longitudes provided only go out to two decimal places, which limits the resolution of the distance calculations to about 0.5 miles. Distances that are very small may not be accurately represented here (e.g. several instances of "0 distance" for distinct Starbucks locations in very close proximity).

In [21]:
# answer goes here
hi_dists = geo_sim_matrix(hi_df)
hi_dists



index,17202,17203,17204,17205,17206,17207,17208,17209,17210,17211,...,17291,17292,17293,17294,17295,17296,17297,17298,17299,17300
index,,,,,,,,,,,,,,,,,,,,,
17202,0.000000,0.690934,0.643386,0.944091,1.460500,6.743576,6.743576,218.428635,218.976944,217.137772,...,109.065346,12.792754,2.373938,16.607107,166.981064,107.264542,98.783943,5.329021,5.675488,6.470637
17203,0.690934,0.000000,0.944121,1.524305,1.888212,6.417289,6.417289,218.067474,218.616730,216.778824,...,108.834190,13.215210,2.050284,16.793444,166.565231,106.962166,98.552231,5.548625,6.120771,6.580589
17204,0.643386,0.944121,0.000000,0.690934,0.944091,6.199704,6.199704,218.976944,219.525787,217.687332,...,109.671234,12.281363,2.921233,15.985024,167.494705,107.842613,99.389603,4.710641,5.179933,5.831356
17205,0.944091,1.524305,0.690934,0.000000,0.643342,6.609524,6.609524,219.338745,219.886642,218.046921,...,109.904938,11.865932,3.304557,15.819399,167.911057,108.146912,99.624100,4.555939,4.747018,5.790081
17206,1.460500,1.888212,0.944091,0.643342,0.000000,6.121677,6.121677,219.886642,220.435089,218.596085,...,110.509941,11.349262,3.826992,15.192343,168.424301,108.724060,100.228790,3.921273,4.240594,5.146738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17296,107.264542,106.962166,107.842613,108.146912,108.724060,111.089050,111.089050,112.120003,112.634476,110.760910,...,12.212746,119.870865,104.923714,123.576645,64.971603,0.000000,14.191094,112.498970,112.833847,113.367599
17297,98.783943,98.552231,99.389603,99.624100,100.228790,103.197449,103.197449,123.401138,123.885117,121.994301,...,10.282104,111.098175,96.501976,115.330681,78.392174,14.191094,0.000000,104.095673,104.204285,105.083298
17298,5.329021,5.548625,4.710641,4.555939,3.921273,4.879154,4.879154,223.542099,224.092758,222.256790,...,114.377487,8.070785,7.597742,11.278437,171.931824,112.498970,104.095673,0.000000,2.170313,1.460423


For each Starbucks, identify its nearest neighboring location in Hawaii (and presumably in the world). Save the output to a dataframe with three columns: Location, Nearest Neighbor, and Distance (Miles).

In [45]:
# answer goes here
nearest_df = pd.DataFrame()
for col in hi_dists.columns:
    temp_df = hi_dists.drop(index=col)
    shortest_dist = temp_df[col].min()
    nearest_location = temp_df[col].idxmin()
    nearest_location
    
    entry = [[col, nearest_location, shortest_dist]]
    
    nearest_df = nearest_df.append(entry)
    
    
    




In [54]:
nearest_df = nearest_df.rename(mapper={0: 'Location', 1: 'Nearest Neighbor', 2: 'Distance (Miles)'}, axis=1)


If the nearest neighbor of a Starbucks location is far away, we could consider that Starbucks to be "on an island". Which five Starbucks in Hawaii are the most "on an island"?

In [56]:
# answer goes here
nearest_df.sort_values(by='Distance (Miles)', ascending=False).head(5)




,Location,Nearest Neighbor,Distance (Miles)
0,17267,17295,15.612447
0,17295,17267,15.612447
0,17278,17284,8.059888
0,17291,17261,7.322419
0,17294,17292,6.739833


In [58]:
five_farthest = nearest_df.sort_values(by='Distance (Miles)', ascending=False).head(5)['Location']

In [59]:
df.iloc[five_farthest]

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
17267,Starbucks,21030-58899,Parker Ranch Center,Company Owned,67-1185 Mamalahoa Highway D108,Kamuela,HI,US,967438427,808-887-6409,GMT-10:00 Pacific/Honolulu,-155.67,20.02
17295,Starbucks,21070-104953,Queens Marketplace,Company Owned,"69-201 Waikoloa Beach Drive, #1001 K-1",Waikoloa,HI,US,967385810,808-886-1888,GMT-10:00 Pacific/Honolulu,-155.88,19.91
17278,Starbucks,21071-107523,Poipu Shopping Village,Company Owned,"2360 Kiahuna Plantation Drive, Suites E-70 & E-80",Koloa,HI,US,967569552,(808) 742-5144,GMT-10:00 Pacific/Honolulu,-159.46,21.88
17291,Starbucks,21040-85520,Pukalani Terrace Center,Company Owned,55 Pukalani Street,Pukalani,HI,US,96768,808-573-5458,GMT-10:00 Pacific/Honolulu,-156.34,20.84
17294,Starbucks,21073-102259,Waianae Mall - Farrington Hwy,Company Owned,"86-120 Farrington Highway, Waikoloa Beach Resort",Waianae,HI,US,967923000,(808) 697-8279,GMT-10:00 Pacific/Honolulu,-158.18,21.44
